### Advanced Data Analysis
Jim Chung
<br>
<br>
I implemented the idea of Michael Utomo that the random forest model can identify the strongest predictor. <br><br> X: 8 selected features <br> y: Grade -  can be changed according to our interests

In [31]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from sklearn import ensemble
from sklearn.preprocessing import OneHotEncoder, Imputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, make_scorer, accuracy_score, classification_report, confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns

import itertools

from tqdm import tqdm

%matplotlib inline

In [2]:
# Data Loaded
data = pd.read_csv("loan.csv")

In [3]:
# See the info
data.describe()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
count,8.873790e+05,8.873790e+05,887379.000000,887379.000000,887379.000000,887379.000000,887379.000000,8.873750e+05,887379.000000,887350.000000,...,21372.000000,18617.000000,21372.000000,21372.000000,21372.000000,21372.000000,8.171030e+05,21372.000000,21372.000000,21372.000000
mean,3.246513e+07,3.500182e+07,14755.264605,14741.877625,14702.464383,13.246740,436.717127,7.502759e+04,18.157039,0.314442,...,36552.811389,71.535317,1.389060,2.975482,5887.979740,60.831939,3.206862e+04,0.943945,1.537058,1.977307
std,2.282734e+07,2.411335e+07,8435.455601,8429.897657,8442.106732,4.381867,244.186593,6.469830e+04,17.190626,0.862244,...,43103.833619,23.022862,1.520129,2.631886,5284.701239,20.013254,3.749826e+04,1.446872,2.723339,2.874067
min,5.473400e+04,7.047300e+04,500.000000,500.000000,0.000000,5.320000,15.670000,0.000000e+00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,-4.000000
25%,9.206643e+06,1.087713e+07,8000.000000,8000.000000,8000.000000,9.990000,260.705000,4.500000e+04,11.910000,0.000000,...,10251.500000,58.600000,0.000000,1.000000,2411.000000,47.700000,1.390000e+04,0.000000,0.000000,0.000000
50%,3.443327e+07,3.709528e+07,13000.000000,13000.000000,13000.000000,12.990000,382.550000,6.500000e+04,17.650000,0.000000,...,24684.500000,74.900000,1.000000,2.000000,4483.000000,61.900000,2.370000e+04,0.000000,0.000000,2.000000
75%,5.490814e+07,5.847135e+07,20000.000000,20000.000000,20000.000000,16.200000,572.600000,9.000000e+04,23.950000,0.000000,...,47858.000000,87.600000,2.000000,4.000000,7771.500000,75.200000,3.980000e+04,1.000000,2.000000,3.000000
max,6.861706e+07,7.354484e+07,35000.000000,35000.000000,35000.000000,28.990000,1445.460000,9.500000e+06,9999.000000,39.000000,...,878459.000000,223.300000,22.000000,43.000000,127305.000000,151.400000,9.999999e+06,17.000000,35.000000,32.000000


In [43]:
# Select the columns to be considered
col = 'loan_amnt,funded_amnt,emp_length,home_ownership,\
annual_inc,last_pymnt_amnt,total_pymnt,last_pymnt_amnt'.split(',')

# We can mention why we should not include the interest rate

X = data[col]
y = data.grade

data[col]

,loan_amnt,funded_amnt,emp_length,home_ownership,annual_inc,last_pymnt_amnt,total_pymnt,last_pymnt_amnt
0,5000.0,5000.0,10+ years,RENT,24000.00,171.62,5861.071414,171.62
1,2500.0,2500.0,< 1 year,RENT,30000.00,119.66,1008.710000,119.66
2,2400.0,2400.0,10+ years,RENT,12252.00,649.91,3003.653644,649.91
3,10000.0,10000.0,10+ years,RENT,49200.00,357.48,12226.302212,357.48
4,3000.0,3000.0,1 year,RENT,80000.00,67.79,3242.170000,67.79
5,5000.0,5000.0,3 years,RENT,36000.00,161.03,5631.377753,161.03
6,7000.0,7000.0,8 years,RENT,47004.00,170.08,8136.840000,170.08
7,3000.0,3000.0,9 years,RENT,48000.00,111.34,3938.144334,111.34
8,5600.0,5600.0,4 years,OWN,40000.00,152.39,646.020000,152.39
9,5375.0,5375.0,< 1 year,RENT,15000.00,121.45,1476.190000,121.45


In [44]:
# Encode the targets: grade
# 7 grades in total
oh = OneHotEncoder()
y = oh.fit_transform(y.factorize(sort=True)[0].reshape(-1,1)).toarray()

In [45]:
# Preprocessing
X['emp_length'] = X['emp_length'].str.extract('(\d+)',)
X['home_ownership'] = X['home_ownership'].factorize(sort=True)[0]

# Missing Value
imputer = Imputer(strategy='median')
X = imputer.fit_transform(X)

# Test set split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43, stratify=y)

In [7]:
'''
# Grid search to find the optimal parameters
param_grid = [
    {'n_estimators': [10, 30], 'max_features': [2, 4, 6]}
]

# Fit random forests
clf = ensemble.RandomForestClassifier()

# accuracy is not a good metric for multi-class problems -> f1 score
scorer = make_scorer(f1_score, average='weighted')
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring=scorer)

# Fit the data
grid_search.fit(X_train, y_train)

# Metrics
cvres = grid_search.cv_results_
for f1_score, params in zip(cvres['f1_score'], cvres['params']):
    print(f1_score, params)
'''

In [ ]:
score = []
param_n = [5, 10, 30, 60]


for n in tqdm(param_n):
    clf = ensemble.RandomForestClassifier(n_estimators=n)
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    score.append(f1_score(y_test, pred, average='weighted'))

sns.set_style("darkgrid")
plt.figure(figsize=(10,10))
plt.plot(range(len(param_n)), score)

 50%|█████     | 2/4 [01:05<01:05, 32.93s/it]

In [ ]:
def plot_confusion_mat(cm, classes, cmap=plt.cm.Blues):
    title='Confusion matrix'

    plt.figure(figsize=(10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    
    tick_marks = np.arange(len(classes))
    
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = 'd'
    thresh = cm.max() / 2.
    
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
def full_report(model, x, y_true, classes, clf='rf'):
    
    if clf == 'rf':
        one_hot = True
    elif clf == 'boosting':
        one_hot = False
    else:
        Exception("Please enter the correct name of the model")
        
    # 2. Predict classes and stores in y_pred
    y_pred = model.predict(x).argmax(axis=1) if one_hot else model.predict(x)
    y_true = y_true.argmax(axis=1) if one_hot else y_true

    
    # 3. Print accuracy score
    print("Accuracy : "+ str(accuracy_score(y_true, y_pred)))
    
    # 4. Print classification report
    print("Classification Report")
    print(classification_report(y_true, y_pred, digits=4))    
    
    # 5. Plot confusion matrix
    cnf_matrix = confusion_matrix(y_true, y_pred)
    
    #print(cnf_matrix)
    plot_confusion_mat(cnf_matrix, classes=classes)    

In [ ]:
full_report(clf, X_test, y_test, [range(7)])

In [ ]:
plt.plot(range(8), clf.feature_importances_)